In [185]:
import sympy as sp

In [186]:
# Pauli matrices
sigma_z = sp.Matrix([[1, 0], [0, -1]])
sigma_x = sp.Matrix([[0, 1], [1, 0]])
sigma_y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
identity = sp.eye(2)

In [187]:
# Define variables
t, omega, omega_0, Omega, phi_0, epsilon, omega_m, theta_m, sig_sym_x, sig_sym_y, sig_sym_z, tau, delta = sp.symbols(
    't omega omega_0 Omega phi_0 epsilon omega_m theta_m sigma_x sigma_y sigma_z tau delta', real=True
)
hbar = sp.Symbol('hbar', real=True)  # Reduced Planck constant

So normally we go to a frame rotating at omega about the z axis - but here we want to go to a frame that removes the phase in the final result.

In [188]:
# Define the unitary matrix U
# This is the simple to omega frame
# U = sp.Matrix([
#     [sp.exp(-sp.I * omega * t / 2), 0],
#     [0, sp.exp(sp.I * omega * t / 2)]
# ])

Define the frame:

In [189]:
# Define H(t)
frame_ham =  hbar*(omega / 2 - (epsilon * omega_m / Omega) * sp.cos(omega_m * t - theta_m)) * sigma_z
frame_ham

Matrix([
[hbar*(omega/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega),                                                                0],
[                                                              0, -hbar*(omega/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)]])

In [190]:
# Define U(t)
U = sp.exp(-(sp.I/hbar)  * sp.integrate(frame_ham, t))
U

Matrix([
[exp(I*(-Omega*omega*t/2 + epsilon*sin(omega_m*t - theta_m))/Omega),                                                                 0],
[                                                                 0, exp(I*(Omega*omega*t/2 - epsilon*sin(omega_m*t - theta_m))/Omega)]])

Yeah this needs to be checked - for pauli 1/2 problems

In [191]:
# Define the lab frame Hamiltonian
# Phase modulated CCD Hamiltonian
cos_argument = omega * t + phi_0 - (2 * epsilon / Omega) * sp.sin(omega_m * t - theta_m)
cos_exp_argument = (sp.exp(sp.I * cos_argument) + sp.exp(-sp.I * cos_argument)) / 2

H_lab = sp.Matrix([
    [ omega_0 / 2, Omega * cos_exp_argument],
    [Omega * cos_exp_argument, -  omega_0 / 2]
])
H_lab

Matrix([
[                                                                                                                                             omega_0/2, Omega*(exp(I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2 + exp(-I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2)],
[Omega*(exp(I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2 + exp(-I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2),                                                                                                                                             -omega_0/2]])

This is the hamiltonian for simple driving

In [192]:
# Construct the Hamiltonian matrix
# gamma, Bx = sp.symbols('gamma Bx')
# H_lab = (hbar * gamma * Bx / 2) * sigma_x * (sp.exp(sp.I * omega * t) + sp.exp(-sp.I * omega * t)) / 2 + sp.Matrix([[hbar * omega_0 / 2, 0], [0, -hbar * omega_0 / 2]])

\begin{equation}
H_{eff} = \left( \hat{U} \hat{H} \hat{U}^\dagger + i\hbar \hat{U} \frac{d\hat{U}^\dagger}{dt} \right) 
\end{equation}

In [193]:
# Compute U^dagger (Hermitian conjugate of U)
U_dagger = U.H
# Compute d(U^dagger)/dt
U_dagger_dt = sp.diff(U_dagger, t)
# Full expression
H = U * H_lab * U_dagger + sp.I * U * U_dagger_dt

In [194]:
# Expand the Hamiltonian
H = H.expand(trig=True)
H

Matrix([
[                                    -omega/2 + omega_0/2 + epsilon*omega_m*sin(theta_m)*sin(omega_m*t)/Omega + epsilon*omega_m*cos(theta_m)*cos(omega_m*t)/Omega, Omega*exp(I*phi_0)/2 + Omega*exp(-I*phi_0)*exp(-2*I*omega*t)*exp(-4*I*epsilon*sin(theta_m)*cos(omega_m*t)/Omega + 4*I*epsilon*sin(omega_m*t)*cos(theta_m)/Omega)/2],
[Omega*exp(I*phi_0)*exp(2*I*omega*t)*exp(4*I*epsilon*sin(theta_m)*cos(omega_m*t)/Omega - 4*I*epsilon*sin(omega_m*t)*cos(theta_m)/Omega)/2 + Omega*exp(-I*phi_0)/2,                                        omega/2 - omega_0/2 - epsilon*omega_m*sin(theta_m)*sin(omega_m*t)/Omega - epsilon*omega_m*cos(theta_m)*cos(omega_m*t)/Omega]])

Make the RWA approximation - sub e^i2wt ->0


In [195]:
H = sp.Matrix(H)

# Simplify Hamiltonian by removing fast oscillating terms
H[0, 1] = H[0, 1].subs(sp.exp(-2 * sp.I * omega * t), 0)
H[1, 0] = H[1, 0].subs(sp.exp(2 * sp.I * omega * t), 0)

In [196]:
# Display the simplified expression
H = sp.simplify(H)
H

Matrix([
[-omega/2 + omega_0/2 + epsilon*omega_m*cos(omega_m*t - theta_m)/Omega,                                                 Omega*exp(I*phi_0)/2],
[                                                Omega*exp(-I*phi_0)/2, omega/2 - omega_0/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega]])

In [197]:
# Simplify Hamiltonian in terms of Pauli matrices
# note the half makes the inner product like thing work for the Pauli matrices
pauli_expression = sp.Rational(1, 2) *(sp.simplify(sp.trace(H.H * sigma_z)) * sig_sym_z + sp.simplify(sp.trace(H.H * sigma_x)) * sig_sym_x + sp.simplify(sp.trace(H.H * sigma_y)) * sig_sym_y)

pauli_expression = pauli_expression.subs(omega_0-omega, delta)
pauli_expression

Omega*sigma_x*cos(phi_0)/2 - Omega*sigma_y*sin(phi_0)/2 + sigma_z*(delta + 2*epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)/2

In [198]:
# Convert to LaTeX string
latex_str = sp.latex(pauli_expression)
print(latex_str)

\frac{\Omega \sigma_{x} \cos{\left(\phi_{0} \right)}}{2} - \frac{\Omega \sigma_{y} \sin{\left(\phi_{0} \right)}}{2} + \frac{\sigma_{z} \left(\delta + \frac{2 \epsilon \omega_{m} \cos{\left(\omega_{m} t - \theta_{m} \right)}}{\Omega}\right)}{2}


The target expression is:

\begin{equation}
H_\mathrm{I} = -\frac{\delta}{2}\sigma_z + \frac{\Omega}{2}\left[\cos(\phi_0)\sigma_x + \sin(\phi_0)\sigma_y\right] + \epsilon_\mathrm{m}\frac{\omega_\mathrm{m}}{\Omega}\cos(\omega_\mathrm{m}t - \theta_\mathrm{m})\sigma_z,
\end{equation}


Currently two things wrong: the -ve sign on the sin and the factors of two?
- the factors of 2 - just from how I was taking inner product of pauli matrices
- - sin????? hmmm

:notably i think this is a matter of the definition of the rotation - I think if I had flipped natural freq and then if I flip the direction of the rotation I get the right sin and cos stuff

SO I think I have to descide on direction convention

### Need to find the frame defining hamiltonian for the next change

In [199]:
frame_ham_2 = hbar*(omega_m/2)* sigma_x
V = sp.exp(-(sp.I/hbar)  * sp.integrate(frame_ham_2, t))

V_dagger = U.H
V_dagger_dt = sp.diff(V_dagger, t)
H = V * H_lab * V_dagger + sp.I * V * V_dagger_dt
H = H.expand(trig=True)
pauli_expression = sp.Rational(1, 2) *(sp.simplify(sp.trace(H.H * sigma_z)) * sig_sym_z + sp.simplify(sp.trace(H.H * sigma_x)) * sig_sym_x + sp.simplify(sp.trace(H.H * sigma_y)) * sig_sym_y)
pauli_expression


sigma_x*(Omega**2*exp(I*(9*Omega*omega*t + 5*Omega*omega_m*t + 6*Omega*phi_0 + 6*epsilon*sin(omega_m*t - theta_m))/(2*Omega)) + Omega**2*exp(I*(9*Omega*omega*t + 7*Omega*omega_m*t + 6*Omega*phi_0 + 6*epsilon*sin(omega_m*t - theta_m))/(2*Omega)) + Omega**2*exp(I*(11*Omega*omega*t + 5*Omega*omega_m*t + 6*Omega*phi_0 + 2*epsilon*sin(omega_m*t - theta_m))/(2*Omega)) + Omega**2*exp(I*(11*Omega*omega*t + 7*Omega*omega_m*t + 6*Omega*phi_0 + 2*epsilon*sin(omega_m*t - theta_m))/(2*Omega)) + Omega*(-omega + omega_0)*exp(I*(11*omega*t + 6*omega_m*t + 8*phi_0 + (Omega*omega_m*t + 2*epsilon*sin(omega_m*t - theta_m))/Omega)/2) + Omega*(omega - omega_0)*exp(I*(11*omega*t/2 + 5*omega_m*t/2 + 4*phi_0 + epsilon*sin(omega_m*t - theta_m)/Omega)) + Omega*(Omega*exp(I*(Omega*omega*t + 2*Omega*phi_0 - 2*epsilon*sin(omega_m*t - theta_m))/(2*Omega)) + Omega*exp(I*(3*Omega*omega*t + 2*Omega*phi_0 - 6*epsilon*sin(omega_m*t - theta_m))/(2*Omega)) - omega*exp(I*(Omega*omega*t - 2*epsilon*sin(omega_m*t - theta_m))/